In [ ]:
import random
import re
import time


# Preperation

In [141]:
### Imports ###
import pandas as pd
import numpy as np

import collections
import itertools
import json
import progressbar

import utils
from tools import shinedalgarno

from IPython.display import clear_output


In [3]:
### Load Variants ###
vdf = pd.read_csv("../data/variants.csv")

### Add a column for randomized 24 nucleotides between two genes
prefix = "CTGCTGGTTCTGGCGAATAGACTAGT"
suffix = "AAGGGCGAGGAGCTCTTTAC"

len_prefix = len(prefix)
len_suffix = len(suffix)

vdf['rand_nucs'] = vdf.variant.str[len_prefix:len_prefix + 24]
vdf['suffix'] = vdf.variant.str[-len_suffix:]

vdf

vdf
# Features Generation

,variant,rand_nucs,suffix
0,CTGCTGGTTCTGGCGAATAGAATAGACTGAACAGGGCGGTCGGCAA...,CTGAACAGGGCGGTCGGCAAAAGA,AAGGGCGAGGAGCTCTTTAC
1,CTGCTGGTTCTGGCGAATAGAATAGACTGAACGCAGGAACAAGAGG...,CTGAACGCAGGAACAAGAGGCGCA,AAGGGCGAGGAGCTCTTTAC
2,CTGCTGGTTCTGGCGAATAGAATAGAGCACCCAGACCATGAGTGAT...,GCACCCAGACCATGAGTGATACAT,AAGGGCGAGGAGCTCTTTAC
3,CTGCTGGTTCTGGCGAATAGAATAGAGTTCACCAAGTGGACTGCAG...,GTTCACCAAGTGGACTGCAGTATA,AAGGGCGAGGAGCTCTTTAC
4,CTGCTGGTTCTGGCGAATAGAATAGATATATAAGTACAAAGTCAGG...,TATATAAGTACAAAGTCAGGTGGT,AAGGGCGAGGAGCTCTTTAC
...,...,...,...
12895,CTGCTGGTTCTGGCGAATAGACTAGTTTTTTTTTACATTGGTTGCA...,TTTTTTTTACATTGGTTGCAAAGA,AAGGGCGAGGAGCTCTTTAC
12896,CTGCTGGTTCTGGCGAATAGACTAGTTTTTTTTTCTCGAGTGATTT...,TTTTTTTTCTCGAGTGATTTGTAA,AAGGGCGAGGAGCTCTTTAC
12897,CTGCTGGTTCTGGCGAATAGACTCGTTGGGAGATGAATTTAAACCG...,TGGGAGATGAATTTAAACCGGAAC,AAGGGCGAGGAGCTCTTTAC
12898,CTGCTGGTTCTGGCGAATAGACTCGTTTGTGATGACGGCAGCCGCA...,TTGTGATGACGGCAGCCGCATCTT,AAGGGCGAGGAGCTCTTTAC


## Codons (3 nucs pair) and 2,4 nucs count in sliding mode with frames (0, 1, 2)
This means that for a sequence: `TGGTTAAAAGCCATGGTCATTGCT`
- There are **2** codons of `AAA`
- There are **3** pairs of `AA`
- There is **1** seq of `AAAA`
*Remember sliding!*

In [138]:



### Generate
count_nucs = np.vectorize(utils.count_nucs_sliding)

for frame in [0, 1, 2]:
  for nucs_count in [2, 3, 4]:
    cdf = pd.Series(count_nucs(vdf.rand_nucs, vdf.variant, nucs_count, frame)).apply(pd.Series).fillna(0)
    vdf = vdf.merge(cdf, on='variant', how='left')

vdf

,AAAA_count_x,AAAA_count1_x,AAAA_count2_x,AAAC_count_x,AAAC_count1_x,AAAC_count2_x,AAAG_count_x,AAAG_count1_x,AAAG_count2_x,AAAT_count_x,...,CCCC_count2_y,GATC_count2_y,CGAC_count2_y,GCCT_count2_y,CTAG_count2_y,TTAA_count2_y,CCTC_count2_y,CGCC_count2_y,TCTC_count2_y,CTCC_count2_y
0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12895,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
12897,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
12898,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


UsageError: Line magic function `%%time` not found.


## Folding strength
when folding is strong at a point in the sequence then its harder for the ribosome to "go through it" and translate it

* Vienna was used to calculate `folds.json`.
We now process it and add it as a feature to the variants.

*!* See `tools/folding.py` to see how `folds.json` was generated.

In [18]:
### Load calculated folds into a DataFrame
with open("../data/folds.json", "r") as f:
  folds_data = json.load(f)

l = []
for var in folds_data:
  variant = var['variant']
  variant_value = var['value']
  d = {
    'variant': variant,
    'variant_fold_level': variant_value
  }
  for i, lv in enumerate(var['local_values']):
    d[f"fold_window_{i}"] = lv
  l.append(d)

fdf = pd.DataFrame(l)

vdf = vdf.merge(fdf, on='variant', how='left')

fdf

,variant,variant_fold_level,fold_window_0,fold_window_1,fold_window_2,fold_window_3,fold_window_4,fold_window_5,fold_window_6,fold_window_7,...,fold_window_30,fold_window_31,fold_window_32,fold_window_33,fold_window_34,fold_window_35,fold_window_36,fold_window_37,fold_window_38,fold_window_39
0,CTGCTGGTTCTGGCGAATAGAATAGACTGAACAGGGCGGTCGGCAA...,-18.4,-3.7,-3.7,-3.7,-3.7,-3.7,-3.7,-3.3,-1.8,...,-2.4,-2.4,-2.4,-4.0,-5.6,-5.6,-5.5,-6.4,-7.4,-8.1
1,CTGCTGGTTCTGGCGAATAGAATAGACTGAACGCAGGAACAAGAGG...,-18.9,-3.7,-3.7,-3.7,-3.7,-3.7,-3.7,-3.7,-3.7,...,-2.8,-2.8,-2.8,-2.8,-2.8,-3.8,-5.5,-6.4,-7.3,-7.8
2,CTGCTGGTTCTGGCGAATAGAATAGAGCACCCAGACCATGAGTGAT...,-15.7,-8.3,-8.1,-7.7,-4.7,-4.7,-4.7,-5.0,-3.8,...,-0.6,-0.4,-0.4,-0.4,-2.0,-3.8,-5.5,-6.3,-6.6,-6.6
3,CTGCTGGTTCTGGCGAATAGAATAGAGTTCACCAAGTGGACTGCAG...,-23.6,-5.1,-5.1,-4.7,-3.7,-5.1,-6.0,-6.0,-6.0,...,-0.6,-0.9,-0.3,-0.6,-2.0,-3.8,-5.5,-6.4,-7.2,-8.6
4,CTGCTGGTTCTGGCGAATAGAATAGATATATAAGTACAAAGTCAGG...,-12.8,-3.7,-3.7,-3.7,-3.7,-3.7,-3.7,-3.3,-1.8,...,-0.3,-0.3,-0.3,-0.3,-2.0,-3.8,-5.5,-6.3,-6.6,-6.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12895,CTGCTGGTTCTGGCGAATAGACTAGTTTTTTTTTACATTGGTTGCA...,-15.3,-6.9,-6.9,-6.5,-5.0,-2.6,-2.1,-2.1,-2.1,...,-1.4,-1.4,-1.4,-1.4,-2.0,-3.8,-5.5,-6.4,-7.4,-8.1
12896,CTGCTGGTTCTGGCGAATAGACTAGTTTTTTTTTCTCGAGTGATTT...,-16.1,-6.9,-6.9,-6.5,-5.0,-2.6,-2.1,-2.1,-2.1,...,-3.0,-4.0,-4.6,-3.5,-5.3,-8.1,-9.2,-9.0,-7.3,-8.0
12897,CTGCTGGTTCTGGCGAATAGACTCGTTGGGAGATGAATTTAAACCG...,-19.6,-6.2,-7.2,-7.8,-7.8,-7.8,-7.8,-7.8,-7.8,...,-1.3,-1.3,-1.3,-1.3,-2.2,-3.8,-5.5,-6.4,-6.7,-6.7
12898,CTGCTGGTTCTGGCGAATAGACTCGTTTGTGATGACGGCAGCCGCA...,-19.0,-2.9,-2.9,-2.9,-3.4,-3.4,-3.5,-4.7,-4.0,...,-6.7,-4.0,-4.0,-4.2,-6.1,-6.4,-6.2,-7.3,-6.6,-6.6


## CAI, TAI and ENC Indexes

In [19]:
## CAI Index -- Prepare references ##
# load data
# odf = pd.read_csv('../data/organism.csv', delimiter='\t')
# odf[['external_id_prefix', 'external_id_suffix']] = odf.string_external_id.str.split('.', 1, expand=True)
# odf
#
# odf.external_id_prefix.value_counts()  ## all values are same -- we can skip this
#
# ecoli_df = pd.read_csv('../data/E_coli_data.csv')  # use e coli genes as a reference
# ecoli_df

## values in organism.csv which are missing from E_coli_data.csv
# missing_from_ref = set(odf['external_id_suffix'].unique()) - set(ecoli_df['gene_id_1'].unique())
# ## values in E_coli_data.csv which are missing from organism.csv
# missing_from_org = set(ecoli_df['gene_id_1'].unique()) - set(odf['external_id_suffix'].unique())
#
# missing_from_ref, missing_from_ref  # none in both, good
#
# ## Let's go with an inner merge
# ecoli_df = odf.merge(ecoli_df, how='inner', left_on='external_id_suffix', right_on='gene_id_1')
# ecoli_df

# take top 20% rows
# df_top_20p = ecoli_df.nlargest(round(0.2 * vdf.shape[0]), 'abundance')
# df_top_20p = df_top_20p.rename(columns={'gene_ORF': 'sequence'})
# df_top_20p
#
# orfs = df_top_20p['sequence']  # reference sequences
# orfs

### Prepare weights ###

# count the number of each codon in the sequences
# sequences = (
#   (sequence[i: i + 3].upper() for i in range(0, len(sequence), 3))
#   for sequence in orfs
# )
# codons = itertools.chain.from_iterable(
#   sequences
# )  # flat list of all codons (to be used for counting)
# counts = collections.Counter(codons)
#
# weights = {}
#
# for codon, count in counts.items():
#   syns_counts = []
#   for c in utils._synonymous_codons.get(codon, []):
#     syns_counts.append(counts.get(c, 0.5))
#
#   weights[codon] = count / max(syns_counts or [1])


# print(collections.Counter(wrap("".join(orfs.tolist()), 3)))
# calculate weights from E COLI reference
# weights
#
# ### calculate CAI ###
# #!! TODO: Fix CAI !!##

# vdf['cai'] = vdf.apply(lambda r: utils.CAI(sequence=r.rand_nucs, weights=weights), axis=1)
# vdf.head(1)[['variant', 'cai']]

# counts


cdf = pd.read_csv('features_ready.csv')
vdf['cai'] = cdf['CAI']
vdf['tai'] = cdf['tAI']
vdf['enc'] = cdf['ENC']

In [20]:
# TAI calculation is the same as CAI, except that in TAI we need to cut the last codon (end codon) when calculating.

wdf = pd.read_csv("../data/Ecoli_TAI_reference_weights.csv")

weights = dict(zip(wdf['codon'].tolist(), wdf['weight'].tolist()))

vdf['tai'] = vdf.apply(lambda r: utils.TAI(sequence=r.rand_nucs[:-3], weights=weights), axis=1)
vdf['tai']

0        0.215934
1        0.237521
2        0.176490
3        0.178358
4        0.158545
           ...   
12895    0.152273
12896    0.177032
12897    0.204294
12898    0.132002
12899    0.204294
Name: tai, Length: 12900, dtype: float64

In [21]:
### ENC Index ###
vdf['enc'] = vdf.apply(lambda r: utils.ENC(sequence=r.rand_nucs), axis=1)
vdf['enc'].value_counts()

32.500000    298
30.000000    284
27.500000    268
28.500000    260
32.000000    244
            ... 
17.250000      1
15.625000      1
21.375000      1
12.541667      1
30.750000      1
Name: enc, Length: 432, dtype: int64

In [ ]:
# vdf['cai']
# pd.concat([cdf['tAI'], vdf['tai']], axis=1)
# cdf['tAI'].corr(vdf['tai'], method='spearman')
# cdf = pd.read_csv('features_ready.csv')
#
# cdf['CAI'].corr(vdf['cai'], method='spearman')
# cdf.head(1)[['variants', 'CAI']]

# cdf['ENC'].corr(vdf['enc'], method='spearman')

## Chimera ARS

In [22]:
# try chimera gui (not working...)
# with open('../data/randnucs.csv', 'w') as fp:
#   fp.write("\n".join(['RandNucs', *df['rand_nucs'].tolist()]))

# load chimera feature from csv
chimera_df = pd.read_csv('../data/Chimera_ARS.csv').rename(columns={"Chimera ARS score": "chimera"})

# add column to df
vdf['chimera'] = chimera_df['chimera']
vdf[['variant', 'rand_nucs', 'chimera']]

,variant,rand_nucs,chimera
0,CTGCTGGTTCTGGCGAATAGAATAGACTGAACAGGGCGGTCGGCAA...,CTGAACAGGGCGGTCGGCAAAAGA,7.500000
1,CTGCTGGTTCTGGCGAATAGAATAGACTGAACGCAGGAACAAGAGG...,CTGAACGCAGGAACAAGAGGCGCA,6.458333
2,CTGCTGGTTCTGGCGAATAGAATAGAGCACCCAGACCATGAGTGAT...,GCACCCAGACCATGAGTGATACAT,6.208333
3,CTGCTGGTTCTGGCGAATAGAATAGAGTTCACCAAGTGGACTGCAG...,GTTCACCAAGTGGACTGCAGTATA,5.833333
4,CTGCTGGTTCTGGCGAATAGAATAGATATATAAGTACAAAGTCAGG...,TATATAAGTACAAAGTCAGGTGGT,6.625000
...,...,...,...
12895,CTGCTGGTTCTGGCGAATAGACTAGTTTTTTTTTACATTGGTTGCA...,TTTTTTTTACATTGGTTGCAAAGA,6.708333
12896,CTGCTGGTTCTGGCGAATAGACTAGTTTTTTTTTCTCGAGTGATTT...,TTTTTTTTCTCGAGTGATTTGTAA,6.125000
12897,CTGCTGGTTCTGGCGAATAGACTCGTTGGGAGATGAATTTAAACCG...,TGGGAGATGAATTTAAACCGGAAC,7.250000
12898,CTGCTGGTTCTGGCGAATAGACTCGTTTGTGATGACGGCAGCCGCA...,TTGTGATGACGGCAGCCGCATCTT,6.708333


## Shine-Dalgarno

In [23]:
# # for shinedalgarno we need to take the rand_nucs of a sequence:
# # CTGAACAGGGCGGTCGGCAAAAGA
# # and cut it to parts of six:
# # CTGAACA, TGAACA, GAACAG, ...
# # when we reach the end of the sequence, and we don't have enough chars to make up 6 letters sequence we need to take extra letters from the end of suffix.
# # after cutting to pieces, get all unique strings and attach them with the `AGGAGG&` prefix. after that calculate using Vienna the values and use a lookup table to match to each variant row (4096 vienna calculations vs ~12900*30)
#
# # so basically an easier way is to take all 4096 combinations:
# combs = itertools.product(['A', 'C', 'G', 'T'], repeat=6)
# inputs = [f'AGGAGG&{"".join(seq)}' for seq in combs]
# inputs
#
# # now send to Vienna...
# with open('../data/shinedalgarno.txt', 'w') as f:
#   f.write("\n".join(parts))
#
# # We now need to run tools/shinedalgarno.py to create the .json file...
# # shinedalgarno.calculate_from_file_and_save_to_json()
#
# # mapping dataframe
with open('../data/shinedalgarno.json') as f:
  shinedalgarno_mapping = json.load(f)

shinedalgarno_mapping = {r['seq'].replace('AGGAGG&', ''): r['value'] for r in shinedalgarno_mapping}


#
#
def split_rand_nucs_to_sixs(rand_nucs, suffix):
  parts = set()

  seq = rand_nucs + suffix  # we use suffix because at the end we don't have enough to fill 6 chars
  for i in range(len(rand_nucs)):
    parts.add(seq[i:i + 6])

  return parts


def apply_map_shinedalgarno_to_parts(row):
  total = 0

  parts = split_rand_nucs_to_sixs(row.rand_nucs, row.suffix)
  for i, part in enumerate(parts):
    val = shinedalgarno_mapping[part]
    total += val
    row[f'shinedal_sliding_{i}'] = val

  row['shindal_mean'] = total / len(parts)

  return row


vdf = vdf.apply(lambda r: apply_map_shinedalgarno_to_parts(r), axis=1).fillna(0)

# Protein Levels
We now need to calculate the protein levels for each variant.
We get new sequences from all_seqs_in_bins.csv file and calculate the protein levels for each variant based on count column multiple grup value.


In [24]:
pdf = pd.read_csv("../data/all_seqs_in_bins.csv")

pdf = pdf.reset_index().rename(columns={"Sample": "group", "seq": "variant"})

pdf = pdf[['variant', 'count', 'group']]

pdf = pdf[pdf.variant.isin(vdf.variant.unique())]

# for each row now we will calculate protein level
# based on its group protein level and count (simply multiply them)
groups_protein_level_mapping = dict(
  P4=185, P5=196, P6=224, P7=243,
  P8=346, P9=488, P10=771, P11=1854
)
pdf['protein_level'] = pdf.apply(lambda r: r['count'] * groups_protein_level_mapping[r['group']], axis=1)
pdf
# now sort by protein levels
pdf = pdf.sort_values(by=['protein_level'], ascending=False)
pdf

,variant,count,group,protein_level
54893,CTGCTGGTTCTGGCGAATAGACTAGTTGGGAGATGAATTTAAACCG...,6243,P10,4813353
45113,CTGCTGGTTCTGGCGAATAGACTAGTTGGGAGATGAATTTAAACCG...,6584,P9,3212992
0,CTGCTGGTTCTGGCGAATAGACTAGTCGCGAGTTCTAGATGGAAGC...,15458,P4,2859730
1,CTGCTGGTTCTGGCGAATAGACTAGTTACATACGGGGGTGAGCGCG...,12542,P4,2320270
54894,CTGCTGGTTCTGGCGAATAGACTAGTCGCGAGTTCTAGATGGAAGC...,2888,P10,2226648
...,...,...,...,...
44443,CTGCTGGTTCTGGCGAATAGAATAGTTGGACATTGATTTCACCTAT...,2,P8,692
44442,CTGCTGGTTCTGGCGAATAGAATAGTTGGACAACCCATGAACGGCA...,2,P8,692
44441,CTGCTGGTTCTGGCGAATAGAATAGTTGCCTGGAGTATTTAATGTT...,2,P8,692
44440,CTGCTGGTTCTGGCGAATAGAATAGTTGATAGACGGCAAAAATAAT...,2,P8,692


# Bins

In [25]:
n = len(pdf) // 8000
tdf = pdf.copy(deep=True)[:n * 8000]
tdf['bin'] = tdf.index // n
tdf

,variant,count,group,protein_level,bin
54893,CTGCTGGTTCTGGCGAATAGACTAGTTGGGAGATGAATTTAAACCG...,6243,P10,4813353,9148
45113,CTGCTGGTTCTGGCGAATAGACTAGTTGGGAGATGAATTTAAACCG...,6584,P9,3212992,7518
0,CTGCTGGTTCTGGCGAATAGACTAGTCGCGAGTTCTAGATGGAAGC...,15458,P4,2859730,0
1,CTGCTGGTTCTGGCGAATAGACTAGTTACATACGGGGGTGAGCGCG...,12542,P4,2320270,0
54894,CTGCTGGTTCTGGCGAATAGACTAGTCGCGAGTTCTAGATGGAAGC...,2888,P10,2226648,9149
...,...,...,...,...,...
42042,CTGCTGGTTCTGGCGAATAGACTAGTGCGTGAGTCTTAAATTTAGG...,6,P8,2076,7007
42043,CTGCTGGTTCTGGCGAATAGACTAGTGCTAAAGTGTCTTAGCTATA...,6,P8,2076,7007
42044,CTGCTGGTTCTGGCGAATAGACTAGTGCTATGTATATTCCCTACCA...,6,P8,2076,7007
42045,CTGCTGGTTCTGGCGAATAGACTAGTGCTCCTTTTATCATCTCCGC...,6,P8,2076,7007


In [26]:
# cut to bins
def cut_df_to_bins(df, bins_count, calculate_mean_protein_level=True):
  n = len(df) // bins_count
  df = df.copy(deep=True)[:n * bins_count]
  df['bin'] = df.index // n  # bins_count

  if calculate_mean_protein_level:
    df['mean_protein_level'] = df.groupby('bin')['protein_level'].transform('mean')

  return df


bins_sizes = [8000, 4000, 2000, 1000, 500, 250]
bins_df = {}  # {8000: 8000 bins DataFrame, 4000: 4000 bins DataFrame, ...}

for bin_size in bins_sizes:
  bins_df[bin_size] = cut_df_to_bins(pdf, bin_size)

In [17]:
### NO NANS ###

In [27]:
# to each row we will add the features of its variant from df
# then we will create a mean value of each feature in each bin

# add only numeric features so we can mean after
features_columns = list(vdf.select_dtypes(['number']).columns)
adf = vdf[['variant', *features_columns]]

adf

,variant,AAAA_count,AAAA_count1,AAAA_count2,AAAC_count,AAAC_count1,AAAC_count2,AAAG_count,AAAG_count1,AAAG_count2,...,shinedal_sliding_3,shinedal_sliding_4,shinedal_sliding_5,shinedal_sliding_6,shinedal_sliding_7,shinedal_sliding_8,shinedal_sliding_9,tai,variant_fold_level_x,variant_fold_level_y
0,CTGCTGGTTCTGGCGAATAGAATAGACTGAACAGGGCGGTCGGCAA...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,-0.2,0.0,-0.2,0.0,0.0,0.215934,-18.4,-18.4
1,CTGCTGGTTCTGGCGAATAGAATAGACTGAACGCAGGAACAAGAGG...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.237521,-18.9,-18.9
2,CTGCTGGTTCTGGCGAATAGAATAGAGCACCCAGACCATGAGTGAT...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-1.3,0.0,0.0,0.0,0.0,0.0,0.176490,-15.7,-15.7
3,CTGCTGGTTCTGGCGAATAGAATAGAGTTCACCAAGTGGACTGCAG...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-1.2,-1.8,0.0,0.0,0.0,-0.6,0.178358,-23.6,-23.6
4,CTGCTGGTTCTGGCGAATAGAATAGATATATAAGTACAAAGTCAGG...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.158545,-12.8,-12.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12895,CTGCTGGTTCTGGCGAATAGACTAGTTTTTTTTTACATTGGTTGCA...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.152273,-15.3,-15.3
12896,CTGCTGGTTCTGGCGAATAGACTAGTTTTTTTTTCTCGAGTGATTT...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-1.9,0.0,0.0,0.0,-0.2,0.177032,-16.1,-16.1
12897,CTGCTGGTTCTGGCGAATAGACTCGTTGGGAGATGAATTTAAACCG...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.204294,-19.6,-19.6
12898,CTGCTGGTTCTGGCGAATAGACTCGTTTGTGATGACGGCAGCCGCA...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-1.5,0.0,0.0,-0.5,-0.9,0.132002,-19.0,-19.0


In [28]:
bins_df[8000]

,variant,count,group,protein_level,bin,mean_protein_level
54893,CTGCTGGTTCTGGCGAATAGACTAGTTGGGAGATGAATTTAAACCG...,6243,P10,4813353,9148,4813353.0
45113,CTGCTGGTTCTGGCGAATAGACTAGTTGGGAGATGAATTTAAACCG...,6584,P9,3212992,7518,3212992.0
0,CTGCTGGTTCTGGCGAATAGACTAGTCGCGAGTTCTAGATGGAAGC...,15458,P4,2859730,0,1798274.0
1,CTGCTGGTTCTGGCGAATAGACTAGTTACATACGGGGGTGAGCGCG...,12542,P4,2320270,0,1798274.0
54894,CTGCTGGTTCTGGCGAATAGACTAGTCGCGAGTTCTAGATGGAAGC...,2888,P10,2226648,9149,1588260.0
...,...,...,...,...,...,...
42042,CTGCTGGTTCTGGCGAATAGACTAGTGCGTGAGTCTTAAATTTAGG...,6,P8,2076,7007,2076.0
42043,CTGCTGGTTCTGGCGAATAGACTAGTGCTAAAGTGTCTTAGCTATA...,6,P8,2076,7007,2076.0
42044,CTGCTGGTTCTGGCGAATAGACTAGTGCTATGTATATTCCCTACCA...,6,P8,2076,7007,2076.0
42045,CTGCTGGTTCTGGCGAATAGACTAGTGCTCCTTTTATCATCTCCGC...,6,P8,2076,7007,2076.0


In [29]:
# now put the features
for bins_count, bdf in bins_df.items():
  assert bdf[bdf.isna().any(axis=1)].shape[0] == 0, "Got NaN"

  bdf = bdf.merge(adf, on='variant', how='left')

  assert bdf[bdf.isna().any(axis=1)].shape[0] == 0, "Got NaN (AFTER MERGE)"

  print(bdf)

  # and calculate mean
  # bdf_new = bdf.copy().drop(features_columns, axis=1)
  bdf_new = pd.DataFrame()

  for feature in features_columns:
    bdf_new['group_' + feature] = bdf.groupby('bin')[feature].mean()

  bdf_new['protein_level_mean'] = bdf.groupby('bin')['protein_level'].mean()
  bdf_new['variant'] = bdf['variant']

  bins_df[bins_count] = bdf_new

bins_df[4000]

                                                 variant  count group  \
0      CTGCTGGTTCTGGCGAATAGACTAGTTGGGAGATGAATTTAAACCG...   6243   P10   
1      CTGCTGGTTCTGGCGAATAGACTAGTTGGGAGATGAATTTAAACCG...   6584    P9   
2      CTGCTGGTTCTGGCGAATAGACTAGTCGCGAGTTCTAGATGGAAGC...  15458    P4   
3      CTGCTGGTTCTGGCGAATAGACTAGTTACATACGGGGGTGAGCGCG...  12542    P4   
4      CTGCTGGTTCTGGCGAATAGACTAGTCGCGAGTTCTAGATGGAAGC...   2888   P10   
...                                                  ...    ...   ...   
47995  CTGCTGGTTCTGGCGAATAGACTAGTGCGTGAGTCTTAAATTTAGG...      6    P8   
47996  CTGCTGGTTCTGGCGAATAGACTAGTGCTAAAGTGTCTTAGCTATA...      6    P8   
47997  CTGCTGGTTCTGGCGAATAGACTAGTGCTATGTATATTCCCTACCA...      6    P8   
47998  CTGCTGGTTCTGGCGAATAGACTAGTGCTCCTTTTATCATCTCCGC...      6    P8   
47999  CTGCTGGTTCTGGCGAATAGACTAGTGCTCGCCTACAAAGAAATTG...      6    P8   

       protein_level   bin  mean_protein_level  AAAA_count  AAAA_count1  \
0            4813353  9148           4813353.0  

C:\Users\nir\AppData\Local\Temp\ipykernel_4460\3099372406.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bdf_new['group_' + feature] = bdf.groupby('bin')[feature].mean()
C:\Users\nir\AppData\Local\Temp\ipykernel_4460\3099372406.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bdf_new['group_' + feature] = bdf.groupby('bin')[feature].mean()
C:\Users\nir\AppData\Local\Temp\ipykernel_4460\3099372406.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times,

                                                 variant  count group  \
0      CTGCTGGTTCTGGCGAATAGACTAGTTGGGAGATGAATTTAAACCG...   6243   P10   
1      CTGCTGGTTCTGGCGAATAGACTAGTTGGGAGATGAATTTAAACCG...   6584    P9   
2      CTGCTGGTTCTGGCGAATAGACTAGTCGCGAGTTCTAGATGGAAGC...  15458    P4   
3      CTGCTGGTTCTGGCGAATAGACTAGTTACATACGGGGGTGAGCGCG...  12542    P4   
4      CTGCTGGTTCTGGCGAATAGACTAGTCGCGAGTTCTAGATGGAAGC...   2888   P10   
...                                                  ...    ...   ...   
51995  CTGCTGGTTCTGGCGAATAGACTAGTAGTTTCTCACTGGACCAGTA...      5    P7   
51996  CTGCTGGTTCTGGCGAATAGACTAGTAGGTCAAGCCATGAAGGACA...      5    P7   
51997  CTGCTGGTTCTGGCGAATAGACTAGTAGGTCGAGCAATTGAGGCTA...      5    P7   
51998  CTGCTGGTTCTGGCGAATAGACTAGTAGGTTGAGGGGGGGCAAGCG...      5    P7   
51999  CTGCTGGTTCTGGCGAATAGACTAGTAGTAAATGAGGTGTGATTTA...      5    P7   

       protein_level   bin  mean_protein_level  AAAA_count  AAAA_count1  \
0            4813353  4222        2.104740e+06  

C:\Users\nir\AppData\Local\Temp\ipykernel_4460\3099372406.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bdf_new['group_' + feature] = bdf.groupby('bin')[feature].mean()
C:\Users\nir\AppData\Local\Temp\ipykernel_4460\3099372406.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bdf_new['group_' + feature] = bdf.groupby('bin')[feature].mean()
C:\Users\nir\AppData\Local\Temp\ipykernel_4460\3099372406.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times,

                                                 variant  count group  \
0      CTGCTGGTTCTGGCGAATAGACTAGTTGGGAGATGAATTTAAACCG...   6243   P10   
1      CTGCTGGTTCTGGCGAATAGACTAGTTGGGAGATGAATTTAAACCG...   6584    P9   
2      CTGCTGGTTCTGGCGAATAGACTAGTCGCGAGTTCTAGATGGAAGC...  15458    P4   
3      CTGCTGGTTCTGGCGAATAGACTAGTTACATACGGGGGTGAGCGCG...  12542    P4   
4      CTGCTGGTTCTGGCGAATAGACTAGTCGCGAGTTCTAGATGGAAGC...   2888   P10   
...                                                  ...    ...   ...   
51995  CTGCTGGTTCTGGCGAATAGACTAGTAGTTTCTCACTGGACCAGTA...      5    P7   
51996  CTGCTGGTTCTGGCGAATAGACTAGTAGGTCAAGCCATGAAGGACA...      5    P7   
51997  CTGCTGGTTCTGGCGAATAGACTAGTAGGTCGAGCAATTGAGGCTA...      5    P7   
51998  CTGCTGGTTCTGGCGAATAGACTAGTAGGTTGAGGGGGGGCAAGCG...      5    P7   
51999  CTGCTGGTTCTGGCGAATAGACTAGTAGTAAATGAGGTGTGATTTA...      5    P7   

       protein_level   bin  mean_protein_level  AAAA_count  AAAA_count1  \
0            4813353  2111        1.200372e+06  

...

                                                 variant  count group  \
0      CTGCTGGTTCTGGCGAATAGACTAGTTGGGAGATGAATTTAAACCG...   6243   P10   
1      CTGCTGGTTCTGGCGAATAGACTAGTTGGGAGATGAATTTAAACCG...   6584    P9   
2      CTGCTGGTTCTGGCGAATAGACTAGTCGCGAGTTCTAGATGGAAGC...  15458    P4   
3      CTGCTGGTTCTGGCGAATAGACTAGTTACATACGGGGGTGAGCGCG...  12542    P4   
4      CTGCTGGTTCTGGCGAATAGACTAGTCGCGAGTTCTAGATGGAAGC...   2888   P10   
...                                                  ...    ...   ...   
52495  CTGCTGGTTCTGGCGAATAGACTAATTCAACGGGATAACTTTCCTC...      2    P8   
52496  CTGCTGGTTCTGGCGAATAGACTAATTCAACGGACCAACTTTCCTC...      2    P8   
52497  CTGCTGGTTCTGGCGAATAGACTAATTACATACGGGGACGAGCGCG...      2    P8   
52498  CTGCTGGTTCTGGCGAATAGACTAATTAAACGGTATCACTTTCTTC...      2    P8   
52499  CTGCTGGTTCTGGCGAATAGACTAGAGTAAATGTGTTGATTACTGG...      2    P8   

       protein_level  bin  mean_protein_level  AAAA_count  AAAA_count1  \
0            4813353  522       233494.058140    

C:\Users\nir\AppData\Local\Temp\ipykernel_4460\3099372406.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bdf_new['group_' + feature] = bdf.groupby('bin')[feature].mean()
C:\Users\nir\AppData\Local\Temp\ipykernel_4460\3099372406.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bdf_new['group_' + feature] = bdf.groupby('bin')[feature].mean()
C:\Users\nir\AppData\Local\Temp\ipykernel_4460\3099372406.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times,

                                                 variant  count group  \
0      CTGCTGGTTCTGGCGAATAGACTAGTTGGGAGATGAATTTAAACCG...   6243   P10   
1      CTGCTGGTTCTGGCGAATAGACTAGTTGGGAGATGAATTTAAACCG...   6584    P9   
2      CTGCTGGTTCTGGCGAATAGACTAGTCGCGAGTTCTAGATGGAAGC...  15458    P4   
3      CTGCTGGTTCTGGCGAATAGACTAGTTACATACGGGGGTGAGCGCG...  12542    P4   
4      CTGCTGGTTCTGGCGAATAGACTAGTCGCGAGTTCTAGATGGAAGC...   2888   P10   
...                                                  ...    ...   ...   
52745  CTGCTGGTTCTGGCGAATAGACCAGTCACTTATTATGTCAATGGGG...      2    P8   
52746  CTGCTGGTTCTGGCGAATAGACCAGCTCGTGGCGAGTTCGCTGGGC...      2    P8   
52747  CTGCTGGTTCTGGCGAATAGACACGTCGCGAGTTCTAGATGGAAGC...      2    P8   
52748  CTGCTGGTTCTGGCGAATAGACAAGTTATGAATAGGCTCGAATATG...      2    P8   
52749  CTGCTGGTTCTGGCGAATAGACAAGTTACTATAAAGTGTGTGTGGT...      2    P8   

       protein_level  bin  mean_protein_level  AAAA_count  AAAA_count1  \
0            4813353  260       352074.165775    

C:\Users\nir\AppData\Local\Temp\ipykernel_4460\3099372406.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bdf_new['group_' + feature] = bdf.groupby('bin')[feature].mean()
C:\Users\nir\AppData\Local\Temp\ipykernel_4460\3099372406.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bdf_new['group_' + feature] = bdf.groupby('bin')[feature].mean()
C:\Users\nir\AppData\Local\Temp\ipykernel_4460\3099372406.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times,

,group_AAAA_count,group_AAAA_count1,group_AAAA_count2,group_AAAC_count,group_AAAC_count1,group_AAAC_count2,group_AAAG_count,group_AAAG_count1,group_AAAG_count2,group_AAAT_count,...,group_shinedal_sliding_5,group_shinedal_sliding_6,group_shinedal_sliding_7,group_shinedal_sliding_8,group_shinedal_sliding_9,group_tai,group_variant_fold_level_x,group_variant_fold_level_y,protein_level_mean,variant
bin,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.810000,-0.330000,0.000000,-0.550000,-0.500000,0.174604,-17.540000,-17.540000,1.269248e+06,CTGCTGGTTCTGGCGAATAGACTAGTTGGGAGATGAATTTAAACCG...
1,0.181818,0.181818,0.181818,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,...,-0.463636,-0.309091,-0.190909,-0.427273,-0.436364,0.181189,-17.590909,-17.590909,5.742400e+05,CTGCTGGTTCTGGCGAATAGACTAGTTGGGAGATGAATTTAAACCG...
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,...,-0.750000,-0.333333,-0.416667,-0.458333,-0.658333,0.179502,-18.808333,-18.808333,4.830658e+05,CTGCTGGTTCTGGCGAATAGACTAGTCGCGAGTTCTAGATGGAAGC...
3,0.000000,0.000000,0.000000,0.100000,0.100000,0.100000,0.000000,0.000000,0.000000,0.100000,...,-0.660000,-0.320000,-1.190000,-0.630000,-0.160000,0.170745,-19.360000,-19.360000,4.222070e+05,CTGCTGGTTCTGGCGAATAGACTAGTTACATACGGGGGTGAGCGCG...
4,0.000000,0.000000,0.000000,0.153846,0.076923,0.076923,0.000000,0.000000,0.000000,0.153846,...,-0.615385,-0.776923,-0.569231,-0.100000,-0.961538,0.192882,-19.238462,-19.238462,3.912892e+05,CTGCTGGTTCTGGCGAATAGACTAGTCGCGAGTTCTAGATGGAAGC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4974,0.153846,0.153846,0.153846,0.076923,0.076923,0.076923,0.000000,0.000000,0.000000,0.076923,...,-0.530769,-0.315385,-0.400000,-0.053846,-0.115385,0.178336,-19.007692,-19.007692,3.855000e+03,CTGCTGGTTCTGGCGAATAGACTAGTATAAACGAGTTTTCAACCCT...
4975,0.076923,0.076923,0.076923,0.000000,0.000000,0.000000,0.153846,0.153846,0.153846,0.076923,...,-0.723077,-0.300000,-0.669231,-0.953846,-0.576923,0.175441,-17.707692,-17.707692,3.855000e+03,CTGCTGGTTCTGGCGAATAGACTAGTCGACATAAAATATTAGAAAG...
4976,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.276923,-0.330769,-0.538462,-0.646154,-0.853846,0.182481,-17.661538,-17.661538,3.855000e+03,CTGCTGGTTCTGGCGAATAGACTAGTATACAGGTGCCGGATGGGTC...


In [30]:
bins_df[8000]

,group_AAAA_count,group_AAAA_count1,group_AAAA_count2,group_AAAC_count,group_AAAC_count1,group_AAAC_count2,group_AAAG_count,group_AAAG_count1,group_AAAG_count2,group_AAAT_count,...,group_shinedal_sliding_5,group_shinedal_sliding_6,group_shinedal_sliding_7,group_shinedal_sliding_8,group_shinedal_sliding_9,group_tai,group_variant_fold_level_x,group_variant_fold_level_y,protein_level_mean,variant
bin,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,...,-1.220000,-0.360000,0.000000,-0.100000,-0.180000,0.175862,-18.580000,-18.580000,1.798274e+06,CTGCTGGTTCTGGCGAATAGACTAGTTGGGAGATGAATTTAAACCG...
1,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,...,-0.400000,-0.300000,0.000000,-1.000000,-0.820000,0.173346,-16.500000,-16.500000,7.402220e+05,CTGCTGGTTCTGGCGAATAGACTAGTTGGGAGATGAATTTAAACCG...
2,0.5,0.5,0.5,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.25,...,-0.950000,-0.650000,-0.475000,-0.450000,-0.975000,0.196303,-16.900000,-16.900000,6.002325e+05,CTGCTGGTTCTGGCGAATAGACTAGTCGCGAGTTCTAGATGGAAGC...
3,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,...,0.000000,-0.050000,-0.033333,-0.483333,-0.150000,0.177699,-18.133333,-18.133333,5.621533e+05,CTGCTGGTTCTGGCGAATAGACTAGTTACATACGGGGGTGAGCGCG...
4,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,...,-1.300000,-0.900000,-0.560000,-0.760000,-0.960000,0.177472,-19.980000,-19.980000,5.154100e+05,CTGCTGGTTCTGGCGAATAGACTAGTCGCGAGTTCTAGATGGAAGC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10782,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,...,-0.350000,-0.166667,-0.450000,-1.216667,-0.550000,0.186108,-17.666667,-17.666667,3.855000e+03,CTGCTGGTTCTGGCGAATAGACTAGTGCTGAGCAATTGTCGGTGTA...
10783,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,...,-0.166667,-0.650000,-1.383333,0.000000,-1.666667,0.180013,-18.966667,-18.966667,3.855000e+03,CTGCTGGTTCTGGCGAATAGACTAGTGGGTGTAGTGCAAACGCTGG...
10784,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,...,-1.275000,-1.125000,-0.475000,-0.550000,-0.500000,0.175083,-20.650000,-20.650000,3.855000e+03,CTGCTGGTTCTGGCGAATAGACTAGTGGTGAACCAGGAGTTCCTTT...


In [31]:
bins_df[8000][bins_df[8000].isna().any(axis=1)]

,group_AAAA_count,group_AAAA_count1,group_AAAA_count2,group_AAAC_count,group_AAAC_count1,group_AAAC_count2,group_AAAG_count,group_AAAG_count1,group_AAAG_count2,group_AAAT_count,...,group_shinedal_sliding_5,group_shinedal_sliding_6,group_shinedal_sliding_7,group_shinedal_sliding_8,group_shinedal_sliding_9,group_tai,group_variant_fold_level_x,group_variant_fold_level_y,protein_level_mean,variant
bin,,,,,,,,,,,,,,,,,,,,,
